# Quantitative Validation of dendritic and somatic excitability in medium spiny neurons, following Figure 2 of Lindroos et al. (2018).


### - Tests dendritic excitability, by examining the local change in calcium concentration as a function of somatic distance, 
###   following a backpropagating action potential. Model output is compared against experimental data from Day et al. (2008).
### Day, M., Wokosin, D., Plotkin, J. L., Tian, X., and Surmeier, D. J. (2008). Differential excitability and modulation of striatal medium spiny neuron dendrites. J. Neurosci. 28, 11603–11614. doi: 10.1523/JNEUROSCI.1840-08.2008


### - Tests somatic excitability, by examining the firing-rate of the neuron as a function of the input current. Model output is
### compared against experimental data from Planert et al. (2013). Eight F-I curves from that paper are considered in this test.
### Planert H, Berger TK, Silberberg G (2013). Membrane properties of striatal direct and indirect pathway neurons in mouse and rat slices and their modulation by dopamine. PLoS ONE 8(3): e57054. doi:10.1371/journal.pone.0057054

### - Validation results are associated to this model entry in the Model Catalog:
### https://model-catalog.brainsimulation.eu/#model_id.7828848c-0cfb-4e35-99c7-6e1c3c560558

## Install the Python-client library for the Validation Service

In [ ]:
!pip uninstall -y hbp_validation_framework
# !pip install hbp_validation_framework
!pip install -U git+https://github.com/HumanBrainProject/hbp-validation-client

# Install some Python packages needed, including the validation test suite 'basalunit'

In [ ]:
# %%script echo skipping

!pip install -q sciunit
!pip install -q bluepyopt
!pip install -q snudda
!pip install joblib
!pip install neuron

In [ ]:
!pip uninstall -y basalunit
!pip install -q git+https://github.com/pedroernesto/basalunit

## Download Lindroos et al.(2018) model and compile the NEURON code if needed

In [ ]:
import os

# !git clone https://github.com/pedroernesto/237653 Model_Lindroos2018
cwd = os.getcwd()
model_dir = os.path.join(cwd, 'Model_Lindroos2018')

# To compile the NEURON code, if needed
'''
os.chdir(model_dir)
!rm -rf x86_64/
!nrnivmodl
os.chdir(cwd)
'''

# Load the model entry from Model Catalog. Create a model object with 'basalunit'.

In [ ]:
from hbp_validation_framework import ModelCatalog, utils
mc = ModelCatalog(username="pedroernesto")
model_alias = "MSND1 full"
model_version = "1.0"

In [ ]:
from basalunit.utils import CellModel_Lindroos2018
model = CellModel_Lindroos2018(model_path=model_dir)
model.model_alias = model_alias
model.model_version = model_version

# Check out first the execution of test for Dendritic Excitability

In [ ]:
from basalunit.tests.test_dendriticExcitability import DendriticExcitability_Test
base_directory = os.path.join(model_dir, './validation_results/DendriticExcitability_Test')
test = DendriticExcitability_Test(model_path=model_dir, base_directory=base_directory)

prediction = test.generate_prediction(model)
observation = test.observation

test.compute_score(observation,prediction)

### Running now the test and registering the results on the Model Catalog

In [ ]:
test_alias = "MSND1_dendritic_excitability"
result, score = utils.run_test(username="pedroernesto", 
                               model=model, 
                               test_alias=test_alias, 
                               storage_collab_id="basal-ganglia-validation",
                               register_result=False)
# print(result["id"] if result else None)
print("https://model-catalog.brainsimulation.eu/#result_id." + result["id"] if result else None)
print(score)

print("====================== Completed ======================")

# Check out first the execution of test for Somatic Excitability

In [ ]:
from basalunit.tests.test_somaticExcitability import SomaticExcitability_Test
base_directory = os.path.join(model_dir, './validation_results/SomaticExcitability_Test')
test = SomaticExcitability_Test(model_path=model_dir, base_directory=base_directory)

prediction = test.generate_prediction(model)
observation = test.observation

test.compute_score(observation,prediction)

### Running now the test against 8 experimental data, and registering the results on the Model Catalog

In [ ]:
for i in range(1,9):
    
    test_alias = "MSND1_somatic_excitability_Planert2013-D1-FI-trace" + str(i)
    result, score = utils.run_test(username="pedroernesto", 
                                   model=model, 
                                   test_alias=test_alias, 
                                   storage_collab_id="basal-ganglia-validation",
                                   register_result=False)
    # print(result["id"] if result else None)
    print("https://model-catalog.brainsimulation.eu/#result_id." + result["id"] if result else None)
    print(score)

    print("====================== Completed ======================")